In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

# Variable declarations
route_d = []
bus_d = []

# route details function
def route(route_details):
    for i in route_details:
        anchor_tag = i.find_element(By.TAG_NAME, 'a')
        route_name = anchor_tag.get_attribute("title")
        route_link = anchor_tag.get_attribute("href")
        route_d.append({'route_name':route_name,'route_link':route_link})
    # print(route_d)

#scroll to next set of RTC cards
def scrolling():
    WebDriverWait(driver, 30)
    scroll = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="Carousel"]/span/span')))
    WebDriverWait(driver, 40)
    input()
    # action = ActionChains(driver)
    # action.move_to_element(scroll).click().perform()
    WebDriverWait(driver, 40)
    return
    
#function to scroll the redbus page
def scroll_Page():
    body = driver.find_element(By.TAG_NAME, "body")
    for _ in range(10):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(10)
    return

#getting rtc route links and name:
def rtc_links(a,b):
    for i in range(a,b):
        body = driver.find_element('tag name', 'body')
        body.send_keys(Keys.PAGE_DOWN)
        WebDriverWait(driver, 40)
        if i>2:
            scrolling()
        Xpath = f"//*[@id='Carousel']/div[{i}]/div[1]/div[1]/div[2]/div[1]/div[1]"
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, Xpath)))
        element.click()
        route_details = driver.find_elements(By.CLASS_NAME, "route_details")
        route(route_details)
        WebDriverWait(driver, 30)
        Home_Button = driver.find_elements(By.XPATH, '//*[@id="root"]/div/div[2]/div/ul/li[1]/a/span')
        Home_Button[0].click()
        WebDriverWait(driver, 40)
    return

#getting all the bus details
def bus_details_scrap(link,name):
    driver = webdriver.Chrome()

    # Open the Redbus website
    driver.get(link)
    WebDriverWait(driver, 10)
    driver.maximize_window()
    try:
        Xpath = '//*[@id="result-section"]/div[1]/div/div[2]/div/div[4]/div[2]'
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(By.XPATH, Xpath))
        element.click()
    except:
        WebDriverWait(driver, 1)
    WebDriverWait(driver, 120)
    
    scroll_Page()
    driver.implicitly_wait(1)
    bus_elements = driver.find_elements(By.CSS_SELECTOR, "div.bus-item")
    
    for bus in bus_elements:
        busname = bus.find_element(By.CSS_SELECTOR, "div.travels.lh-24.f-bold.d-color").text 
        bustype = bus.find_element(By.CSS_SELECTOR, "div.bus-type.f-12.m-top-16.l-color.evBus").text
        departing_time = bus.find_element(By.CSS_SELECTOR, "div.dp-time.f-19.d-color.f-bold").text
        duration = bus.find_element(By.CSS_SELECTOR, "div.dur.l-color.lh-24").text
        reaching_time = bus.find_element(By.CSS_SELECTOR, "div.bp-time.f-19.d-color.disp-Inline").text
        try:
            star_rating = bus.find_element(By.CSS_SELECTOR, "div.rating-sec.lh-24 > div > span").text
        except:
            star_rating = "None"
        star_rating = float(star_rating) if star_rating != "None" else 0.0
        price = bus.find_element(By.CSS_SELECTOR, "span.f-19.f-bold").text
        price = float(price.replace('₹', '').replace(',', '').strip())
        driver.implicitly_wait(1)
        try:
            seats_available = bus.find_element(By.XPATH, "//div[@class='seat-left m-top-30']").text
        except:
            seats_available = bus.find_element(By.XPATH, "//div[@class='seat-left m-top-16']").text
        seats_available = int(seats_available.split()[0])
        bus_d.append({'Route_Name': name,
                      'Route_link': link,
                      'Bus_Name': busname,
                      'Bus_Type': bustype,
                      'Departing_Time': departing_time, 
                      'Duration': duration, 
                      'Reaching_Time': reaching_time, 
                      'Star_Rating': star_rating, 
                      'Price': price, 
                      'Seats_Availability': seats_available})
    print("Scraped Bus Details:")

# Setup the ChromeDriver
driver = webdriver.Chrome()

# Open the Redbus website
driver.get("https://www.redbus.in/")
WebDriverWait(driver, 10)
driver.maximize_window()

#route links for APRTC and KERELA
rtc_links(1,11)

#converting list to dataframe and storing in csv file
df = pd.DataFrame(route_d)
output_path = 'C:/Users/Sathiaseelan/Documents/Redbus_project/route_details.csv'
df.to_csv(output_path)

#bus_details for all the links
for route in route_d:
    link = route.get("route_link")
    name = route.get("route_name")
    bus_details_scrap(link,name)

#converting list to dataframe and storing in csv file
df = pd.DataFrame(bus_d)
output_path = 'C:/Users/Sathiaseelan/Documents/Redbus_project/bus_details.csv'
df.to_csv(output_path)

In [97]:
#Connecting to Mysql for data storage
import pandas as pd
import pymysql

myconnection = pymysql.connect(host='127.0.0.1',user='root',passwd='Mainboot@1')

# myconnection.cursor().execute("create database Redbus")

df = pd.read_csv("C:/Users/Sathiaseelan/Documents/Redbus_project/bus_details.csv")
df.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)
df['Departing_Time'] = pd.to_datetime(df['Departing_Time'], format='%H:%M').dt.strftime('%H:%M:%S')
df['Reaching_Time'] = pd.to_datetime(df['Reaching_Time'], format='%H:%M').dt.strftime('%H:%M:%S')

columns = ",".join(f"{i} {j}"
for i,j in zip(df.columns,df.dtypes)).replace("float64","float").replace("datetime64[ns]","TIME").replace("int64","int").replace("object","text")
columns
columns = columns.replace("ID int", "ID INT PRIMARY KEY") 
table_name = "Bus_Routes" 
myconnection.cursor().execute(f"create table Redbus.{table_name}({columns})")
for i in range(len(df)):
    myconnection.cursor().execute(f"insert into Redbus.{table_name} values {tuple(df.iloc[i])}")
    myconnection.commit()

#altering datatype in mysql to time
myconnection.cursor().execute("use Redbus")
myconnection.cursor().execute("ALTER TABLE bus_routes MODIFY COLUMN Departing_Time TIME")
myconnection.cursor().execute("ALTER TABLE bus_routes MODIFY COLUMN Reaching_Time TIME")
myconnection.commit()

In [1]:
#Connecting to Mysql for data storage
import pandas as pd
import pymysql

myconnection = pymysql.connect(host='127.0.0.1',user='root',passwd='Mainboot@1')

# myconnection.cursor().execute("create database Redbus")

df = pd.read_csv("C:/Users/Sathiaseelan/Documents/Redbus_project/route_details.csv")
# df.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)
# df['Departing_Time'] = pd.to_datetime(df['Departing_Time'], format='%H:%M').dt.strftime('%H:%M:%S')
# df['Reaching_Time'] = pd.to_datetime(df['Reaching_Time'], format='%H:%M').dt.strftime('%H:%M:%S')

columns = ",".join(f"{i} {j}"
for i,j in zip(df.columns,df.dtypes)).replace("float64","float").replace("datetime64[ns]","TIME").replace("int64","int").replace("object","text")
columns
columns = columns.replace("ID int", "ID INT PRIMARY KEY") 
table_name = "Bus_Routes_State" 
myconnection.cursor().execute(f"create table Redbus.{table_name}({columns})")
for i in range(len(df)):
    myconnection.cursor().execute(f"insert into Redbus.{table_name} values {tuple(df.iloc[i])}")
    myconnection.commit()

# #altering datatype in mysql to time
# myconnection.cursor().execute("use Redbus")
# myconnection.cursor().execute("ALTER TABLE bus_routes MODIFY COLUMN Departing_Time TIME")
# myconnection.cursor().execute("ALTER TABLE bus_routes MODIFY COLUMN Reaching_Time TIME")
# myconnection.commit()